In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized, 0.01)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized, 0.01)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized, 0.01)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized, 0.01)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 3)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-Additive_Depth3-.01/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.24352523883846072
Average test loss: 0.011312417642109924
Epoch 2/300
Average training loss: 0.06594387014706929
Average test loss: 0.009837653728822867
Epoch 3/300
Average training loss: 0.05952252914177047
Average test loss: 0.009221684822605715
Epoch 4/300
Average training loss: 0.05597749098141988
Average test loss: 0.009063741586688491
Epoch 5/300
Average training loss: 0.05431898597213957
Average test loss: 0.01001914536456267
Epoch 6/300
Average training loss: 0.052557628197802436
Average test loss: 0.008771701109078196
Epoch 7/300
Average training loss: 0.05080292812983195
Average test loss: 0.008849304373065631
Epoch 8/300
Average training loss: 0.04971153234110938
Average test loss: 0.008515717866520087
Epoch 9/300
Average training loss: 0.04872854452000724
Average test loss: 0.008272285001145468
Epoch 10/300
Average training loss: 0.047995075249009665
Average test loss: 0.008426337146096759
Epoch 1

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 22.43
Average PSNR for Projection Layer 1 across 2500 images: 24.24
Average PSNR for Projection Layer 2 across 2500 images: 25.11
Average PSNR for Projection Layer 3 across 2500 images: 25.69
Average PSNR for Projection Layer 4 across 2500 images: 26.05
Average PSNR for Projection Layer 0 across 2500 images: 23.46
Average PSNR for Projection Layer 1 across 2500 images: 26.04
Average PSNR for Projection Layer 2 across 2500 images: 27.37
Average PSNR for Projection Layer 3 across 2500 images: 28.08
Average PSNR for Projection Layer 4 across 2500 images: 28.69
Average PSNR for Projection Layer 0 across 2500 images: 24.10
Average PSNR for Projection Layer 1 across 2500 images: 27.06
Average PSNR for Projection Layer 2 across 2500 images: 28.85
Average PSNR for Projection Layer 3 across 2500 images: 29.77
Average PSNR for Projection Layer 4 across 2500 images: 30.38
Average PSNR for Projection Layer 0 across 2500 images: 24.44
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-Additive_Depth3-.01/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 2.3951824896600513
Average test loss: 0.01321575334171454
Epoch 2/300
Average training loss: 0.3154480259153578
Average test loss: 0.010011888128187922
Epoch 3/300
Average training loss: 0.23307043857044643
Average test loss: 0.009560091531111134
Epoch 4/300
Average training loss: 0.20012644267082214
Average test loss: 0.008690088629722596
Epoch 5/300
Average training loss: 0.1818781664106581
Average test loss: 0.008399052269756794
Epoch 6/300
Average training loss: 0.1698397096792857
Average test loss: 0.009098480845491091
Epoch 7/300
Average training loss: 0.16109314845667946
Average test loss: 0.008018135566678313
Epoch 8/300
Average training loss: 0.1549092503918542
Average test loss: 0.008542229372594091
Epoch 9/300
Average training loss: 0.14806197877724966
Average test loss: 0.007512800840040048
Epoch 10/300
Average training loss: 0.14304465646213954
Average test loss: 0.007418435414218241
Epoch 11/300
A

In [10]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 21.94
Average PSNR for Projection Layer 1 across 2500 images: 23.69
Average PSNR for Projection Layer 2 across 2500 images: 24.43
Average PSNR for Projection Layer 3 across 2500 images: 25.07
Average PSNR for Projection Layer 4 across 2500 images: 25.39
Average PSNR for Projection Layer 5 across 2500 images: 25.56
Average PSNR for Projection Layer 6 across 2500 images: 25.80
Average PSNR for Projection Layer 7 across 2500 images: 26.13
Average PSNR for Projection Layer 8 across 2500 images: 26.28
Average PSNR for Projection Layer 9 across 2500 images: 26.42
Average PSNR for Projection Layer 10 across 2500 images: 26.50
Average PSNR for Projection Layer 11 across 2500 images: 26.59
Average PSNR for Projection Layer 12 across 2500 images: 26.63
Average PSNR for Projection Layer 13 across 2500 images: 26.72
Average PSNR for Projection Layer 14 across 2500 images: 26.68
Average PSNR for Projection Layer 0 across 2500 images: 22.90
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-Additive_Depth3-.01/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 15.784444282107883
Average test loss: 0.01412928224603335
Epoch 2/300
Average training loss: 3.05816351975335
Average test loss: 0.010887865845527913
Epoch 3/300
Average training loss: 2.138427090220981
Average test loss: 0.010027557996412119
Epoch 4/300
Average training loss: 1.592377396053738
Average test loss: 0.010320795993424125
Epoch 5/300
Average training loss: 1.2269281118181017
Average test loss: 0.00903927997002999
Epoch 6/300
Average training loss: 0.9284771104918585
Average test loss: 0.008536077882680628
Epoch 7/300
Average training loss: 0.7243667730225457
Average test loss: 0.008089195192688041
Epoch 8/300
Average training loss: 0.5951408725844489
Average test loss: 0.0076583137934406595
Epoch 9/300
Average training loss: 0.5086848589579265
Average test loss: 0.0078066546875569555
Epoch 10/300
Average training loss: 0.4472273894680871
Average test loss: 0.007543494718770186
Epoch 11/300
Average t

In [12]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 20.14
Average PSNR for Projection Layer 1 across 2500 images: 22.26
Average PSNR for Projection Layer 2 across 2500 images: 23.03
Average PSNR for Projection Layer 3 across 2500 images: 23.69
Average PSNR for Projection Layer 4 across 2500 images: 24.30
Average PSNR for Projection Layer 5 across 2500 images: 24.53
Average PSNR for Projection Layer 6 across 2500 images: 25.04
Average PSNR for Projection Layer 7 across 2500 images: 25.18
Average PSNR for Projection Layer 8 across 2500 images: 25.41
Average PSNR for Projection Layer 9 across 2500 images: 25.59
Average PSNR for Projection Layer 10 across 2500 images: 25.79
Average PSNR for Projection Layer 11 across 2500 images: 25.91
Average PSNR for Projection Layer 12 across 2500 images: 26.08
Average PSNR for Projection Layer 13 across 2500 images: 26.16
Average PSNR for Projection Layer 14 across 2500 images: 26.11
Average PSNR for Projection Layer 15 across 2500 images: 26.29
Av